In [1]:
from halotools.sim_manager import CachedHaloCatalog, FakeSim
from halotools.empirical_models import PrebuiltHodModelFactory, Zheng07Cens, Zheng07Sats, TrivialPhaseSpace, NFWPhaseSpace, HodModelFactory
from halotools.mock_observables import return_xyz_formatted_array
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from Corrfunc.theory.wp import wp
import MCMC_data_file
from numpy.linalg import inv
from iminuit import Minuit,describe,minimize
from iminuit.util import make_func_code
%pylab inline
import donemusic

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Populating the interactive namespace from numpy and matplotlib


In [2]:
class LeastSquares: 
    def __init__(self, model, knowns):
        self.model = model 
        self.wp_vals, self.ng, self.invcov, self.ng_cov, self.bin_edges = knowns
         
    def __call__(self, logMmin, sigma_logM, alpha, logM0, logM1):  # par are a variable number of model parameters
        #wp_diff, ng_diff = self.model(self.wp_vals, self.ng, self.invcov, self.ng_cov, self.bin_edges, *par)
        wp_calc_val,ng_calc_val= self.model(logMmin, sigma_logM, alpha, logM0, logM1)
        wp_diff = self.wp_vals - wp_calc_val
        ng_diff = self.ng - ng_calc_val
        chi2 = -0.5 * np.dot(wp_diff, np.dot(self.invcov, wp_diff)) - 0.5 * (ng_diff**2)/(self.ng_cov**2)
        return chi2

In [3]:
wp_ng_vals = MCMC_data_file.get_wp()
bin_edges = MCMC_data_file.get_bins()
bin_cen = (bin_edges[1:]+bin_edges[:-1])/2.
wp_vals = wp_ng_vals[1:len(wp_ng_vals)]
ng = wp_ng_vals[0]
cov_matrix = MCMC_data_file.get_cov()
invcov = inv(cov_matrix[1:len(cov_matrix),1:len(cov_matrix)])
ng_cov = cov_matrix[0,0]
cens_occ_model = Zheng07Cens()
cens_prof_model = TrivialPhaseSpace()

sats_occ_model =  Zheng07Sats()
sats_prof_model = NFWPhaseSpace()

In [4]:
def wp_calc_val(logMmin, sigma_logM, alpha, logM0, logM1):
    
    pi_max = 60.
    Lbox = 250.

    cens_occ_model.param_dict['logMmin'] = logMmin
    cens_occ_model.param_dict['sigma_logM'] = sigma_logM
    sats_occ_model.param_dict['alpha'] = alpha
    sats_occ_model.param_dict['logM0'] = logM0
    sats_occ_model.param_dict['logM1'] = logM1

    model_instance = HodModelFactory(centrals_occupation = cens_occ_model, centrals_profile = cens_prof_model, 
                                 satellites_occupation = sats_occ_model, satellites_profile = sats_prof_model)

    halocat = CachedHaloCatalog(fname='/Users/lmezini/.astropy/cache/halotools/halo_catalogs/bolplanck/rockstar/hlist_1.00231.list.halotools_v0p4.hdf5',update_cached_fname = True) 
    halocat.redshift=0.
    model_instance.populate_mock(halocat) 
    mvir_table = model_instance.mock.halo_table

    pos = return_xyz_formatted_array(model_instance.mock.galaxy_table['x'], model_instance.mock.galaxy_table['y'],
                                 model_instance.mock.galaxy_table['z'],period = Lbox)
    x = pos[:,0]
    y = pos[:,1]
    z = pos[:,2]
    velz = model_instance.mock.galaxy_table['vz']
    pos_zdist = return_xyz_formatted_array(x,y,z, period = Lbox, velocity=velz, velocity_distortion_dimension='z')
    

    wp_calc = wp(Lbox,pi_max,1,bin_edges,pos_zdist[:,0],pos_zdist[:,1],pos_zdist[:,2],verbose=True,
                 xbin_refine_factor=3, ybin_refine_factor=3, zbin_refine_factor=2)
    
    wp_diff = wp_vals-wp_calc['wp']
    #ng_diff = ng -  model_instance.mock.number_density
    ng_calc = model_instance.mock.number_density
    return wp_calc['wp'],ng_calc

In [5]:
guess = [11.96, 0.38,1.16,12.23,13.28]
lsq = LeastSquares(wp_calc_val, [wp_vals, ng, invcov, ng_cov, bin_edges])

In [12]:
m = Minuit(lsq, forced_parameters=('logMmin', 'sigma_logM', 'alpha', 'logM0', 'logM1'),
           logMmin=11.96, sigma_logM=0.38,alpha=1.16,logM0=12.23,logM1=13.28,
           limit_sigma_logM=(0.001,0.6),limit_logMmin=(10.,13.),limit_alpha=(0.5,2.0),
           limit_logM0=(10.5,13.0),limit_logM1=(12.0,14.0),
           error_sigma_logM=0.025,error_logMmin=0.1,error_alpha=0.05,
           error_logM0=0.1,error_logM1=0.1,errordef=1)

In [13]:
m.get_param_states()

------------------------------------------------------------------------------------------------
|   | Name       |   Value   | Hesse Err | Minos Err- | Minos Err+ | Limit-  | Limit+  | Fixed |
------------------------------------------------------------------------------------------------
| 0 | logMmin    |   11.96   |   0.10    |            |            |   10    |   13    |       |
| 1 | sigma_logM |   0.380   |   0.025   |            |            |  0.001  |   0.6   |       |
| 2 | alpha      |   1.16    |   0.05    |            |            |   0.5   |    2    |       |
| 3 | logM0      |   12.23   |   0.10    |            |            |  10.5   |   13    |       |
| 4 | logM1      |   13.28   |   0.10    |            |            |   12    |   14    |       |
------------------------------------------------------------------------------------------------

In [14]:
m.migrad()
donemusic.nonstop()

In gridlink_index_particles_float> Running with [nmesh_x, nmesh_y, nmesh_z]  = 23,23,8.  Time taken =   0.008 sec
0%.........10%.........20%.........30%.........40%.........50%.........60%.........70%.........80%.........90%.........100% done. Time taken =  2.056 secs
In gridlink_index_particles_float> Running with [nmesh_x, nmesh_y, nmesh_z]  = 23,23,8.  Time taken =   0.007 sec
0%.........10%.........20%.........30%.........40%.........50%.........60%.........70%.........80%.........90%.........100% done. Time taken =  1.730 secs
In gridlink_index_particles_float> Running with [nmesh_x, nmesh_y, nmesh_z]  = 23,23,8.  Time taken =   0.009 sec
0%.........10%.........20%.........30%.........40%.........50%.........60%.........70%.........80%.........90%.........100% done. Time taken =  2.406 secs
In gridlink_index_particles_float> Running with [nmesh_x, nmesh_y, nmesh_z]  = 23,23,8.  Time taken =   0.009 sec
0%.........10%.........20%.........30%.........40%.........50%.........60%.....

0%.........10%.........20%.........30%.........40%.........50%.........60%.........70%.........80%.........90%.........100% done. Time taken =  2.007 secs
In gridlink_index_particles_float> Running with [nmesh_x, nmesh_y, nmesh_z]  = 23,23,8.  Time taken =   0.008 sec
0%.........10%.........20%.........30%.........40%.........50%.........60%.........70%.........80%.........90%.........100% done. Time taken =  2.003 secs
In gridlink_index_particles_float> Running with [nmesh_x, nmesh_y, nmesh_z]  = 23,23,8.  Time taken =   0.008 sec
0%.........10%.........20%.........30%.........40%.........50%.........60%.........70%.........80%.........90%.........100% done. Time taken =  1.974 secs
In gridlink_index_particles_float> Running with [nmesh_x, nmesh_y, nmesh_z]  = 23,23,8.  Time taken =   0.008 sec
0%.........10%.........20%.........30%.........40%.........50%.........60%.........70%.........80%.........90%.........100% done. Time taken =  2.028 secs
In gridlink_index_particles_float> Run

In gridlink_index_particles_float> Running with [nmesh_x, nmesh_y, nmesh_z]  = 23,23,8.  Time taken =   0.008 sec
0%.........10%.........20%.........30%.........40%.........50%.........60%.........70%.........80%.........90%.........100% done. Time taken =  2.010 secs
In gridlink_index_particles_float> Running with [nmesh_x, nmesh_y, nmesh_z]  = 23,23,8.  Time taken =   0.008 sec
0%.........10%.........20%.........30%.........40%.........50%.........60%.........70%.........80%.........90%.........100% done. Time taken =  2.047 secs
In gridlink_index_particles_float> Running with [nmesh_x, nmesh_y, nmesh_z]  = 23,23,8.  Time taken =   0.009 sec
0%.........10%.........20%.........30%.........40%.........50%.........60%.........70%.........80%.........90%.........100% done. Time taken =  2.001 secs
In gridlink_index_particles_float> Running with [nmesh_x, nmesh_y, nmesh_z]  = 23,23,8.  Time taken =   0.008 sec
0%.........10%.........20%.........30%.........40%.........50%.........60%.....

0%.........10%.........20%.........30%.........40%.........50%.........60%.........70%.........80%.........90%.........100% done. Time taken =  2.017 secs
In gridlink_index_particles_float> Running with [nmesh_x, nmesh_y, nmesh_z]  = 23,23,8.  Time taken =   0.008 sec
0%.........10%.........20%.........30%.........40%.........50%.........60%.........70%.........80%.........90%.........100% done. Time taken =  2.000 secs
In gridlink_index_particles_float> Running with [nmesh_x, nmesh_y, nmesh_z]  = 23,23,8.  Time taken =   0.008 sec
0%.........10%.........20%.........30%.........40%.........50%.........60%.........70%.........80%.........90%.........100% done. Time taken =  2.006 secs
In gridlink_index_particles_float> Running with [nmesh_x, nmesh_y, nmesh_z]  = 23,23,8.  Time taken =   0.008 sec
0%.........10%.........20%.........30%.........40%.........50%.........60%.........70%.........80%.........90%.........100% done. Time taken =  2.008 secs
In gridlink_index_particles_float> Run


Done


..100% done. Time taken =  2.000 secs


In [15]:
print(m.get_fmin())
m.get_param_states()


------------------------------------------------------------------
| FCN = -3.445E+06              |     Ncalls=119 (119 total)     |
| EDM = NAN (Goal: 1E-05)       |            up = 1.0            |
------------------------------------------------------------------
|  Valid Min.   | Valid Param.  | Above EDM | Reached call limit |
------------------------------------------------------------------
|     True      |     True      |   False   |       False        |
------------------------------------------------------------------
| Hesse failed  |   Has cov.    | Accurate  | Pos. def. | Forced |
------------------------------------------------------------------
|     False     |     True      |   False   |   False   |  True  |
------------------------------------------------------------------


ValueError: cannot convert float NaN to integer

ValueError: cannot convert float NaN to integer

In [16]:
m.matrix(correlation=True)

ValueError: cannot convert float NaN to integer

ValueError: cannot convert float NaN to integer

In [ ]:
m.matrix()  #covariance

In [ ]:
m.draw_contour('logMmin', 'sigma_logM')